# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
# import libraries
import sys
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + 'HWDEdf', echo=False)
import sqlite3
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# load data from database
df=pd.read_sql_table('HWDEdf', con=engine)
df.head()
#yes!

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#let's perform some cleaning
#all we need, is the message column, and the categories
#drop the unnecessary ones, but in case the final datasets in IDE somewhat differs in shape let's keep the columns we want instead
df=df[['message',  'related', 'request','offer', 'aid_related', 'medical_help', 'medical_products','search_and_rescue', 'security', 'military', 'child_alone', 'water','food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees','death', 'other_aid', 'infrastructure_related', 'transport','buildings', 'electricity', 'tools', 'hospitals', 'shops','aid_centers', 'other_infrastructure', 'weather_related', 'floods','storm', 'fire', 'earthquake', 'cold', 'other_weather','direct_report']]

In [5]:
df.head()
#good, exactly what we want

,message,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [75]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [76]:
#test
tokenize("i LOVE kfc!")
#lovely!

['i', 'love', 'kfc']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [79]:
def pipeline():
    pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)), #initial vectorizer to transform data into a 'dictionary'
                    ('tfidf', TfidfTransformer()), #tfidf bit
                    ('clf',MultiOutputClassifier(estimator= RandomForestClassifier())) #classifier that puts the string into one of the multiple(36) cats
                     ])

    
    return pipeline 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [67]:
#define independent variables and dependent variable
category_names = df[['aid_centers', 'aid_related', 'buildings', 'child_alone', 'clothing','cold', 'death', 'direct_report', 'earthquake', 'electricity', 'fire','floods', 'food', 'hospitals', 'infrastructure_related','medical_help', 'medical_products',  'military','missing_people', 'money', 'offer', 'other_aid','other_infrastructure', 'other_weather', 'refugees', 'related','request', 'search_and_rescue', 'security', 'shelter', 'shops', 'storm','tools', 'transport', 'water', 'weather_related']].columns
X=df['message'].values

In [71]:
y=df[category_names]
y.head()

,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
#split the data into test and train, split 30%/70%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [61]:
X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

(18350, 7865, 18350, 7865)

In [78]:

model=pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [80]:
#let's train the pipeline using the built pipeline and training data
model=pipeline()
model.fit(X_train, y_train)
#test
y_pred = model.predict(X_test)
print(classification_report(y_test.values, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

           aid_centers       0.00      0.00      0.00       102
           aid_related       0.75      0.52      0.62      3281
             buildings       0.83      0.12      0.22       388
           child_alone       0.00      0.00      0.00         0
              clothing       0.64      0.06      0.11       121
                  cold       0.79      0.07      0.13       150
                 death       0.76      0.11      0.20       328
         direct_report       0.79      0.31      0.45      1562
            earthquake       0.87      0.66      0.75       694
           electricity       1.00      0.02      0.04       152
                  fire       0.00      0.00      0.00        81
                floods       0.88      0.33      0.48       622
                  food       0.81      0.33      0.47       891
             hospitals       0.00      0.00      0.00        89
infrastructure_related       0.20      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [77]:
#let's train the pipeline using the built pipeline and training data
def pipeline_GS():
    pipeline_GS = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(estimator= RandomForestClassifier()))
                    ])

    parameters = {
                    'vect__min_df':[1,10,50],
                    'tfidf__smooth_idf': [True, False],
                    }
    model  = GridSearchCV(pipeline_GS, param_grid=parameters, cv=3) 
    return model 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [81]:
#let's train the pipeline using the built pipeline and training data
model=pipeline_GS()
model.fit(X_train, y_train)
#test
y_pred = model.predict(X_test)
print(classification_report(y_test.values, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

           aid_centers       0.00      0.00      0.00       102
           aid_related       0.74      0.58      0.65      3281
             buildings       0.72      0.16      0.27       388
           child_alone       0.00      0.00      0.00         0
              clothing       0.78      0.26      0.39       121
                  cold       0.45      0.07      0.12       150
                 death       0.78      0.30      0.44       328
         direct_report       0.79      0.37      0.50      1562
            earthquake       0.89      0.74      0.81       694
           electricity       0.60      0.04      0.07       152
                  fire       0.50      0.02      0.05        81
                floods       0.89      0.45      0.60       622
                  food       0.83      0.53      0.65       891
             hospitals       0.00      0.00      0.00        89
infrastructure_related       0.27      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

# I tried to incorporate the starting verb method in the lessons but for some reason keeps getting an 'list index out of range' error. Given that the model is performing very well already i think i'll pass for now. If anyone can share some insight as to how to make this work that would be most helpful

In [32]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [33]:
def pipeline_FU():
    pipeline_FU = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(estimator= RandomForestClassifier()))
    ])
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
       )
    }

    
    model_GSFU  = GridSearchCV(pipeline_FU, param_grid=parameters, cv=3) 
    return model_GSFU

In [34]:
model_GSFU=pipeline_FU()
model_GSFU.fit(X_train, y_train)
from sklearn.utils.multiclass import type_of_target
type_of_target(y_test)
type_of_target(y_pred)
y_pred = model_GSFU.predict(X_test)
target_names=np.unique(y_pred)
print(classification_report(np.hstack(y_test),np.hstack(y_pred)))

IndexError: list index out of range

### 9. Export your model as a pickle file

In [82]:
def save_model(model, model_filepath):

    with open(model_filepath, 'wb') as pkl_file:
                  pickle.dump(model, pkl_file)
    pkl_file.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [83]:
save_model('model', 'classifier.pkl')